# Supervised data preparation for ML approaches of recession prediction
- Dataframe index: Date (Monthly from 1971-01 to 2023-02)
- Features: values from each econ index
- Labels: recession within M months (M=0, 6, 12)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

## 1. Date-Recession Data
- Dataframe index: Date (Monthly from 1971-01 to 2023-02)
- Labels: recession within M months (M=0, 6, 12)

In [5]:
df_rec = pd.DataFrame(pd.date_range(start='1952-10-01', end='2023-03-01', freq='M'), columns=['DATE'])
df_rec.set_index(df_rec['DATE'], inplace=True)
df_rec.head()

,DATE
DATE,
1952-10-31,1952-10-31
1952-11-30,1952-11-30
1952-12-31,1952-12-31
1953-01-31,1953-01-31
1953-02-28,1953-02-28


In [6]:
recession_periods = [('1953-07-01', '1954-05-01'), ('1957-08-01', '1958-04-01'), ('1960-04-01', '1961-02-01'), ('1969-12-01', '1970-11-01'), 
                     ('1973-11-01', '1975-03-01'), ('1980-01-01', '1980-07-01'), ('1981-07-01', '1982-11-01'), ('1990-07-01', '1991-03-01'),
                     ('2001-03-01', '2001-11-01'), ('2007-12-01', '2009-06-01'), ('2020-02-01', '2020-04-01')]

def is_recession(date):
  for period in recession_periods:
    if date >= pd.to_datetime(period[0]) + pd.DateOffset(months=-THRES_MONTH) and date <= pd.to_datetime(period[1]):
      return 1
  return 0

THRES_MONTH = 0 # we will consider recession within THRES_MONTH months
df_rec['Recession'] = df_rec['DATE'].map(is_recession)
THRES_MONTH = 6
df_rec['Recession_within_6mo'] = df_rec['DATE'].map(is_recession)
THRES_MONTH = 12
df_rec['Recession_within_12mo'] = df_rec['DATE'].map(is_recession)

df_rec['1973-04-01':'1974-01-01']

,DATE,Recession,Recession_within_6mo,Recession_within_12mo
DATE,,,,
1973-04-30,1973-04-30,0,0,1
1973-05-31,1973-05-31,0,1,1
1973-06-30,1973-06-30,0,1,1
1973-07-31,1973-07-31,0,1,1
1973-08-31,1973-08-31,0,1,1
1973-09-30,1973-09-30,0,1,1
1973-10-31,1973-10-31,0,1,1
1973-11-30,1973-11-30,1,1,1
1973-12-31,1973-12-31,1,1,1


In [7]:
df_rec.drop(columns=['DATE'], inplace=True)
df_rec.head()

,Recession,Recession_within_6mo,Recession_within_12mo
DATE,,,
1952-10-31,0,0,1
1952-11-30,0,0,1
1952-12-31,0,0,1
1953-01-31,0,1,1
1953-02-28,0,1,1


In [8]:
# Save data to a csv file
df_rec.to_csv('../../Datasets/date-recession-monthly.csv')

## 2. Date-Recession with feature values (econ indices)
- Dataframe index: Date (Monthly from 1976-06 to 2023-02)
- Features: values from each econ index
    - For now, we excluded index 2 (New Orders) and 8 (S&P500) because they have too little data for monthly period (starts from 1992-02)
- Labels: recession within M months (M=0, 6, 12)

| Index Number |Index|DataType[unit]|Frequency|Availability|
| -- | -- | -- | -- | -- |
| 1 | Inverted Yield Curve | float [Percent] | Daily | June 1976 - April 2023 |
| 2 | New Orders | int [Million Dollars] | Monthly | February 1992 - December 2022 |
| 3 | Jobless Claims | int [Claims] | Weekly | January 1967 - February 2023 |
| 4 | Building Permit | int [Thousand units] | Monthly | January 1960 - January 2023 |
| 5 | Credit Subindex | float [Billion Dollars] | Weekly | January 1971 - May 2023 |
| 6 | Consumer Sentiment | float [none] | Monthly | November 1952 - December 2022 |
| 7 | Wage and Salary Growth | float [Billion Dollars] | Monthly | January 1959 - December 2022 |
| 8 | S&P500 | float [none] | Daily | May 2013 - May 2023 |

In [9]:
# Load date-recession data
df_rec = pd.read_csv('../../Datasets/date-recession-monthly.csv')
df_rec['DATE'] = pd.to_datetime(df_rec['DATE'])
df_rec.set_index(df_rec['DATE'], inplace=True)
df_rec.head()

,DATE,Recession,Recession_within_6mo,Recession_within_12mo
DATE,,,,
1952-10-31,1952-10-31,0,0,1
1952-11-30,1952-11-30,0,0,1
1952-12-31,1952-12-31,0,0,1
1953-01-31,1953-01-31,0,1,1
1953-02-28,1953-02-28,0,1,1


In [10]:
## still だめ since it needs prior preproc for each data of econ index
# def join_data(df_main, file_name, col_name, index_num):
#     df = pd.read_csv('../../Datasets/' + file_name + '.csv') # Dataset file
#     df['DATE'] = pd.to_datetime(df['DATE'])
#     df.set_index(df['DATE'], drop=True, inplace=True)

#     df_monthly = pd.DataFrame(df[col_name].resample('M').mean())
#     col_name_rev = 'F'+str(index_num)+'-'+col_name
#     df_monthly = pd.DataFrame(data={'DATE': df_monthly.index, col_name_rev: df_monthly[col_name]})
    
#     df_main = pd.concat([df_main, df_monthly], join='inner', axis=1)
#     df_main.drop(columns=['DATE'], inplace=True)

#     return df_main

### Index 1 (Inverted Yield Curve)

In [11]:
col_name = 'T10Y2Y'
index_num = 1

df = pd.read_csv('../../Datasets/T10Y2Y.csv') # Dataset file
df['DATE'] = pd.to_datetime(df['DATE'])
df.set_index(df['DATE'], inplace=True)
df[col_name] = pd.to_numeric(df[col_name], errors='coerce')
df.head()

,DATE,T10Y2Y
DATE,,
1976-06-01,1976-06-01,0.68
1976-06-02,1976-06-02,0.71
1976-06-03,1976-06-03,0.70
1976-06-04,1976-06-04,0.77
1976-06-07,1976-06-07,0.79


In [12]:
df_monthly = pd.DataFrame(df[col_name].resample('M').mean())
col_name_rev = 'F'+str(index_num)+'-'+col_name
df_monthly = pd.DataFrame(data={'DATE': df_monthly.index, col_name_rev: df_monthly[col_name]})
# df_monthly.head()

df_rec = pd.concat([df_rec, df_monthly], join='inner', axis=1) # OR df_rec.merge(df_monthly, how='inner', left_index=True, right_index=True)
df_rec.drop(columns=['DATE'], inplace=True)
df_rec.head()

,Recession,Recession_within_6mo,Recession_within_12mo,F1-T10Y2Y
DATE,,,,
1976-06-30,0,0,0,0.801364
1976-07-31,0,0,0,0.981905
1976-08-31,0,0,0,1.137273
1976-09-30,0,0,0,1.178095
1976-10-31,0,0,0,1.427000


### Index 3 (Jobless Claims)

In [13]:
col_name = 'IC4WSA'
index_num = 3
df = pd.read_csv('../../Datasets/IC4WSA.csv') # Dataset file
df['DATE'] = pd.to_datetime(df['DATE'])
df.set_index(df['DATE'], inplace=True)
df.head()

,DATE,IC4WSA
DATE,,
1967-01-28,1967-01-28,209000
1967-02-04,1967-02-04,211000
1967-02-11,1967-02-11,216500
1967-02-18,1967-02-18,219500
1967-02-25,1967-02-25,229000


In [14]:
df_monthly = pd.DataFrame(df[col_name].resample('M').mean())
col_name_rev = 'F'+str(index_num)+'-'+col_name
df_monthly = pd.DataFrame(data={'DATE': df_monthly.index, col_name_rev: df_monthly[col_name]})
# df_monthly.head()

df_rec = pd.concat([df_rec, df_monthly], join='inner', axis=1) # OR df_rec.merge(df_monthly, how='inner', left_index=True, right_index=True)
df_rec.drop(columns=['DATE'], inplace=True)
df_rec.head()

,Recession,Recession_within_6mo,Recession_within_12mo,F1-T10Y2Y,F3-IC4WSA
DATE,,,,,
1976-06-30,0,0,0,0.801364,394375.0
1976-07-31,0,0,0,0.981905,391200.0
1976-08-31,0,0,0,1.137273,388125.0
1976-09-30,0,0,0,1.178095,401625.0
1976-10-31,0,0,0,1.427000,408750.0


### Index 4 (Building Permit)

In [15]:
col_name = 'PERMIT'
index_num = 4
df = pd.read_excel('../../Datasets/PERMIT.xls') # Dataset file
# df['DATE'] = pd.to_datetime(df['observation_date'])
df['DATE'] = df['observation_date']
df.drop(columns=['observation_date'], inplace=True)
df.set_index(df['DATE'], inplace=True)

df.head()

,PERMIT,DATE
DATE,,
1960-01-01,1092,1960-01-01
1960-02-01,1088,1960-02-01
1960-03-01,955,1960-03-01
1960-04-01,1016,1960-04-01
1960-05-01,1052,1960-05-01


In [16]:
df_monthly = pd.DataFrame(df[col_name].resample('M').mean())
col_name_rev = 'F'+str(index_num)+'-'+col_name
df_monthly = pd.DataFrame(data={'DATE': df_monthly.index, col_name_rev: df_monthly[col_name]})
# df_monthly.head()

df_rec = pd.concat([df_rec, df_monthly], join='inner', axis=1) # OR df_rec.merge(df_monthly, how='inner', left_index=True, right_index=True)
df_rec.drop(columns=['DATE'], inplace=True)
df_rec.head()

,Recession,Recession_within_6mo,Recession_within_12mo,F1-T10Y2Y,F3-IC4WSA,F4-PERMIT
DATE,,,,,,
1976-06-30,0,0,0,0.801364,394375.0,1188.0
1976-07-31,0,0,0,0.981905,391200.0,1245.0
1976-08-31,0,0,0,1.137273,388125.0,1309.0
1976-09-30,0,0,0,1.178095,401625.0,1481.0
1976-10-31,0,0,0,1.427000,408750.0,1425.0


### Index 5 (Credit Subindex)

In [17]:
col_name = 'NFCICREDIT'
index_num = 5
df = pd.read_csv('../../Datasets/index-5-NFCICREDIT.csv') # Dataset file
df['DATE'] = pd.to_datetime(df['DATE'])
df.set_index(df['DATE'], drop=True, inplace=True)
df.head()

,DATE,NFCICREDIT
DATE,,
1971-01-08,1971-01-08,-1.90407
1971-01-15,1971-01-15,-1.90886
1971-01-22,1971-01-22,-1.91133
1971-01-29,1971-01-29,-1.91241
1971-02-05,1971-02-05,-1.91255


In [18]:
df_monthly = pd.DataFrame(df['NFCICREDIT'].resample('M').mean())
col_name_rev = 'F'+str(index_num)+'-'+col_name
df_monthly = pd.DataFrame(data={'DATE': df_monthly.index, col_name_rev: df_monthly['NFCICREDIT']})
df_monthly.head()

,DATE,F5-NFCICREDIT
DATE,,
1971-01-31,1971-01-31,-1.909167
1971-02-28,1971-02-28,-1.913507
1971-03-31,1971-03-31,-1.936220
1971-04-30,1971-04-30,-2.026352
1971-05-31,1971-05-31,-2.150960


In [19]:
df_rec = pd.concat([df_rec, df_monthly], join='inner', axis=1) # OR df_rec.merge(df_monthly, how='inner', left_index=True, right_index=True)
df_rec.drop(columns=['DATE'], inplace=True)
df_rec.head()

,Recession,Recession_within_6mo,Recession_within_12mo,F1-T10Y2Y,F3-IC4WSA,F4-PERMIT,F5-NFCICREDIT
DATE,,,,,,,
1976-06-30,0,0,0,0.801364,394375.0,1188.0,-2.229077
1976-07-31,0,0,0,0.981905,391200.0,1245.0,-2.052324
1976-08-31,0,0,0,1.137273,388125.0,1309.0,-1.824368
1976-09-30,0,0,0,1.178095,401625.0,1481.0,-1.611755
1976-10-31,0,0,0,1.427000,408750.0,1425.0,-1.406606


### Index 6 (Consumer Sentiment)
- lots of NaNs ...

In [20]:
col_name = 'UMCSENT'
index_num = 6
df = pd.read_csv('../../Datasets/UMCSENT.csv') # Dataset file
df['DATE'] = pd.to_datetime(df['DATE'])
df.set_index(df['DATE'], inplace=True)
df[col_name] = pd.to_numeric(df[col_name], errors='coerce')

df.head()

,DATE,UMCSENT
DATE,,
1952-11-01,1952-11-01,86.2
1952-12-01,1952-12-01,NaN
1953-01-01,1953-01-01,NaN
1953-02-01,1953-02-01,90.7
1953-03-01,1953-03-01,NaN


In [21]:
df_monthly = pd.DataFrame(df[col_name].resample('M').mean())
col_name_rev = 'F'+str(index_num)+'-'+col_name
df_monthly = pd.DataFrame(data={'DATE': df_monthly.index, col_name_rev: df_monthly[col_name]})
# df_monthly.head()

df_rec = pd.concat([df_rec, df_monthly], join='inner', axis=1) # OR df_rec.merge(df_monthly, how='inner', left_index=True, right_index=True)
df_rec.drop(columns=['DATE'], inplace=True)
df_rec.head()

,Recession,Recession_within_6mo,Recession_within_12mo,F1-T10Y2Y,F3-IC4WSA,F4-PERMIT,F5-NFCICREDIT,F6-UMCSENT
DATE,,,,,,,,
1976-06-30,0,0,0,0.801364,394375.0,1188.0,-2.229077,NaN
1976-07-31,0,0,0,0.981905,391200.0,1245.0,-2.052324,NaN
1976-08-31,0,0,0,1.137273,388125.0,1309.0,-1.824368,89.7
1976-09-30,0,0,0,1.178095,401625.0,1481.0,-1.611755,NaN
1976-10-31,0,0,0,1.427000,408750.0,1425.0,-1.406606,NaN


### Index 7 (Wage and Salary Growth)

In [22]:
col_name = 'A576RC1'
index_num = 7
df = pd.read_csv('../../Datasets/Wage_growth.csv') # Dataset file
df['DATE'] = pd.to_datetime(df['DATE'])
df.set_index(df['DATE'], inplace=True)

df.head()

,DATE,A576RC1
DATE,,
1959-01-01,1959-01-01,252.3
1959-02-01,1959-02-01,253.6
1959-03-01,1959-03-01,256.1
1959-04-01,1959-04-01,258.6
1959-05-01,1959-05-01,260.8


In [23]:
df_monthly = pd.DataFrame(df[col_name].resample('M').mean())
col_name_rev = 'F'+str(index_num)+'-'+col_name
df_monthly = pd.DataFrame(data={'DATE': df_monthly.index, col_name_rev: df_monthly[col_name]})
# df_monthly.head()

df_rec = pd.concat([df_rec, df_monthly], join='inner', axis=1) # OR df_rec.merge(df_monthly, how='inner', left_index=True, right_index=True)
df_rec.drop(columns=['DATE'], inplace=True)
df_rec.head()

,Recession,Recession_within_6mo,Recession_within_12mo,F1-T10Y2Y,F3-IC4WSA,F4-PERMIT,F5-NFCICREDIT,F6-UMCSENT,F7-A576RC1
DATE,,,,,,,,,
1976-06-30,0,0,0,0.801364,394375.0,1188.0,-2.229077,NaN,893.8
1976-07-31,0,0,0,0.981905,391200.0,1245.0,-2.052324,NaN,900.6
1976-08-31,0,0,0,1.137273,388125.0,1309.0,-1.824368,89.7,909.6
1976-09-30,0,0,0,1.178095,401625.0,1481.0,-1.611755,NaN,915.1
1976-10-31,0,0,0,1.427000,408750.0,1425.0,-1.406606,NaN,920.5


### Final Data

In [24]:
df_rec.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 560 entries, 1976-06-30 to 2023-01-31
Freq: M
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Recession              560 non-null    int64  
 1   Recession_within_6mo   560 non-null    int64  
 2   Recession_within_12mo  560 non-null    int64  
 3   F1-T10Y2Y              560 non-null    float64
 4   F3-IC4WSA              560 non-null    float64
 5   F4-PERMIT              560 non-null    float64
 6   F5-NFCICREDIT          560 non-null    float64
 7   F6-UMCSENT             547 non-null    float64
 8   F7-A576RC1             560 non-null    float64
dtypes: float64(6), int64(3)
memory usage: 43.8 KB


In [25]:
df_rec.to_csv('../../Datasets/date-recession-monthly-6-indices.csv')

## Data Preprocessing
To-do:
- Normalization